# Parcels
> This publication inspired by [nbdev](http://nbdev.fast.ai/tutorial/) and [nbConvert](https://nbconvert.readthedocs.io/en/latest/).

- toc: true
- title: Parcels
- template: article
- description: this isnt working
- image: images/company_logo.png
- keywords: thisworks
- badges: true
- comments: true
- categories: [test]
- hide: false
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Get Parcels

In [11]:
#hide 
#cd /workspaces/CVMiniGames.github.io/blog/data

"""
#hide 
import sys
sys.path.append('/workspaces/CVMiniGames.github.io/blog/')
sys.path.append('/workspaces/CVMiniGames.github.io/docs/datasets/')
"""

Grab all parcels

In [2]:
#hide_input 
import requests
import json
url = 'https://www.cryptovoxels.com/api/parcels/search.json'
r = requests.get(url)
data = json.loads(r.text)['parcels']

In [3]:
#hide_input 
import pandas as pd
df = pd.json_normalize(data=data)
# df = df[df['island'] == 'Scarcity'] # Only Scarcity Island
#df = df[df.columns[0:8]] # We don't need all the data

In [4]:
#hide_input 
df.head()

,id,height,address,suburb,island,name,area,distance,owner,owner_name,...,x2,y1,label,y2,z1,z2,geometry.type,geometry.crs.type,geometry.crs.properties.name,geometry.coordinates
0,6748,6,31b Fault Line,Bronze,Scarcity,None,96,1849.130877,0x09d06dcef22051477cde12cfbfc674f60a53c66c,None,...,-1466,8,None,6,-1139,-1127,Polygon,name,EPSG:3857,"[[[-14.74, -11.389899995], [-14.74, -11.27], [..."
1,6747,9,37b Ruby Street,Silver,Scarcity,RUBY LOFT,120,1763.588144,0x3b97b6cbbfcfff88cf4a175e665c5715a3efe4f0,sexman.eth,...,-1257,8,None,9,-1245,-1237,Polygon,name,EPSG:3857,"[[[-12.72, -12.45], [-12.72, -12.37], [-12.570..."
2,6746,9,23b Antarcticite Road,Gold,Scarcity,None,104,1627.018697,0xc6786a54882ca1591745d748a2a92813e35c1a5d,Artolli,...,-1242,8,None,9,-1059,-1051,Polygon,name,EPSG:3857,"[[[-12.55, -10.51], [-12.420100005, -10.51], [..."
3,6745,7,11b Arsenopyrite Ave,Bronze,Scarcity,None,153,1945.814879,0x13816f65ba8195768e6825eb774fd3c6a05a0ac8,None,...,-1527,8,None,7,-1223,-1206,Polygon,name,EPSG:3857,"[[[-15.270100005, -12.23], [-15.36, -12.23], [..."
4,6744,8,7b Analcime Street,Silver,Scarcity,None,165,1778.345861,0xc6786a54882ca1591745d748a2a92813e35c1a5d,Artolli,...,-1317,8,None,8,-1206,-1195,Polygon,name,EPSG:3857,"[[[-13.17, -11.95], [-13.17, -12.06], [-13.319..."


## Save Parcels

In [5]:
cd ../data

/workspaces/3Diot.github.io/blog/data


In [6]:
df[:13].head(3)

,id,height,address,suburb,island,name,area,distance,owner,owner_name,...,x2,y1,label,y2,z1,z2,geometry.type,geometry.crs.type,geometry.crs.properties.name,geometry.coordinates
0,6748,6,31b Fault Line,Bronze,Scarcity,None,96,1849.130877,0x09d06dcef22051477cde12cfbfc674f60a53c66c,None,...,-1466,8,None,6,-1139,-1127,Polygon,name,EPSG:3857,"[[[-14.74, -11.389899995], [-14.74, -11.27], [..."
1,6747,9,37b Ruby Street,Silver,Scarcity,RUBY LOFT,120,1763.588144,0x3b97b6cbbfcfff88cf4a175e665c5715a3efe4f0,sexman.eth,...,-1257,8,None,9,-1245,-1237,Polygon,name,EPSG:3857,"[[[-12.72, -12.45], [-12.72, -12.37], [-12.570..."
2,6746,9,23b Antarcticite Road,Gold,Scarcity,None,104,1627.018697,0xc6786a54882ca1591745d748a2a92813e35c1a5d,Artolli,...,-1242,8,None,9,-1059,-1051,Polygon,name,EPSG:3857,"[[[-12.55, -10.51], [-12.420100005, -10.51], [..."


In [ ]:
import requests
import json

deets = []
for index, row in df.tail(200).iterrows():
  url = 'https://www.cryptovoxels.com/api/parcels/'+str(row['id'])+'.json' 
  r = requests.get(url)
  data = json.loads(r.text) 
  with open('./parcels/'+str(row['id'])+'.json', 'w') as file:
    json.dump(data, file)
  print(row['id'])

## Filtering Parcels

In [52]:
import json
deets = []
for index, row in df[1:2].iterrows():
  url = './parcels/'+str(row['id'])+'.json'
  data = ''
  with open(url, 'r') as j: 
    data = json.loads(j.read()) 
    data = data['parcel'] 
  #coords = data.geometry.coordinates
  #data['coords'] = coords
  #del data.geometry.coordinates
  #//content = data.content.features
  #del data.content.features
  deets.append( data )